In [1]:
from __future__ import print_function, division
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import data as dt
import model_4_o as md
import copy
from pytorch_ssim import ssim
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt
from torchvision import transforms, utils
# e/d + i
# best mode so far

In [2]:
# Load model
cwd = os.getcwd();
path = os.path.join(cwd, 'Saved model', 'encoderresinfo.pth')
model = md.Net()
model.load_state_dict(torch.load(path))
model.eval()

Net(
  (block0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
  )
  (block1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.1)
  )
  (info1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
  )
  (block2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

In [3]:
# print gpu
torch.cuda.set_device(0)
currentDevice = torch.cuda.current_device()
print("Current GPU: " + str(currentDevice))
print(str(torch.cuda.device_count()))
print(str(torch.cuda.get_device_capability(currentDevice)))
print(torch.__version__)

Current GPU: 0
8
(7, 0)
1.0.0


In [4]:
USE_GPU = 1
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = "cpu"
print(device)

cuda:0


In [ ]:
CSVFILE = 0
# generate csv file, run only for the first time
if CSVFILE:
    dt.generate_csv()
# dt.generate_csv()

In [5]:
csvFilePath = dt.get_csv_path()
transformed_dataset = dt.HE_SHG_Dataset(csv_file=csvFilePath,
                                               transform=dt.Compose([                                              
                                               dt.Rescale(96),                                     
                                               dt.Normalize(),
                                               dt.ToTensor()
                                           ]))
# TODO: change the normalization parameters

In [6]:
# batchsize 1200
dataloader = DataLoader(transformed_dataset, batch_size=32,
                        shuffle=True, num_workers=0)

In [ ]:
dt.show_patch(dataloader) 

In [7]:
# declare MSE loss here
criterionMSE = nn.MSELoss()
criterionMSE = criterionMSE.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [8]:
def train(epoch, p, windowsize):
    epoch_loss = 0
    for iteration, batch in enumerate(dataloader):
        input, target = batch['input'].to(device), batch['output'].to(device)

        optimizer.zero_grad()
        output = model(input)
        targetf = target.float()
        targetf = targetf[:, None]
        
        lossMSE = criterionMSE(output, targetf)      
        lossSSIM = 1-ssim(output, targetf, window_size=windowsize)
        
        # 0.75->0.4 after 3 epochs
        loss = p*lossMSE + (1-p)*lossSSIM
        combineLoss = p*lossMSE.item() + (1-p)*lossSSIM.item()
        
        epoch_loss = epoch_loss + combineLoss
        loss.backward()
        optimizer.step()
    
        if iteration%50 == 0:
            print("lossMSE: " + str(lossMSE.item()) +
                  " " + "lossSSIM: " + str(lossSSIM.item()))
            print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(dataloader), loss.item()))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(dataloader)))


In [9]:
def test():
    avg_psnr = 0
    with torch.no_grad():
        for iteration, batch in enumerate(dataloader):
            input, target = batch['input'].to(device), batch['output'].to(device)
                
            prediction = model(input)

            target = target.float()

            outdataloader = {'input':prediction,'output':target}
            
            print(outdataloader['input'].size(), 
                      outdataloader['output'].size())

            plt.figure()
            input_batch, label_batch = outdataloader['input'], outdataloader['output']
            batch_size = 32
            im_size = input_batch.size(2)
            label_batch=label_batch.reshape([batch_size,1,im_size,im_size])
            print(label_batch.size())
#             for img in input_batch:
#                 for t, m, s in zip(img, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]):
#                     t.mul_(s).add_(m)
                            
#             for img in label_batch:
#                 for t, m, s in zip(img, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]):
#                     t.mul_(s).add_(m)                           
            grid = utils.make_grid(input).cpu()
            plt.imshow(grid.numpy().transpose((1, 2, 0)))
            plt.figure()
    
            grid = utils.make_grid(input_batch).cpu()
            plt.imshow(grid.numpy().transpose((1, 2, 0)))
            plt.figure()

            grid = utils.make_grid(label_batch).cpu()
            plt.imshow(grid.numpy().transpose((1, 2, 0)))

            plt.axis('off')
            plt.ioff()
            plt.show()
            
            targetf = target[:, None]
            
            lossMSE = criterionMSE(prediction, targetf)      
            lossSSIM = -ssim(prediction, targetf)
        
            p = 0.25
            loss = p*lossMSE + (1-p)*lossSSIM
            combineLoss = p*lossMSE.item() + (1-p)*lossSSIM.item()
#             mse = criterion(prediction, target.float())

            psnr = 10 * torch.log10(1 / loss)
            avg_psnr += psnr
            if iteration == 16:
                break
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(dataloader)))

In [ ]:
SAVE = 0
if SAVE:
    cwd = os.getcwd();
    path = os.path.join(cwd, 'Saved model', 'encoderresinfo.pth')
    torch.save(model.state_dict(), path)

In [ ]:
# choose GPU and load model to GPU
gpu_ids = [1, 2, 3, 4, 5, 6]
torch.cuda.set_device(gpu_ids[0])
model = torch.nn.DataParallel(model, device_ids=gpu_ids)
model.cuda()

In [ ]:
model.to(device)
l = 0.01
p = 0
windowsize = 16
for epoch in range(1, 200 + 1):
    if epoch%5 == 0:
        windowsize = windowsize+1
        p = p*1
        l = l*0.95
        if windowsize >= 16:
            windowsze = 16
    optimizer = optim.Adam(model.parameters(), lr=l)
        
    train(epoch, p, windowsize)

lossMSE: 0.003795218886807561 lossSSIM: 0.23527216911315918
===> Epoch[1](0/25343): Loss: 0.2353
lossMSE: 0.0034746204037219286 lossSSIM: 0.2395954728126526
===> Epoch[1](50/25343): Loss: 0.2396
lossMSE: 0.005337276495993137 lossSSIM: 0.37588900327682495
===> Epoch[1](100/25343): Loss: 0.3759
lossMSE: 0.00580750172957778 lossSSIM: 0.25722736120224
===> Epoch[1](150/25343): Loss: 0.2572
lossMSE: 0.003364729695022106 lossSSIM: 0.2693172097206116
===> Epoch[1](200/25343): Loss: 0.2693
lossMSE: 0.005230859387665987 lossSSIM: 0.24197548627853394
===> Epoch[1](250/25343): Loss: 0.2420
lossMSE: 0.006105458363890648 lossSSIM: 0.302550733089447
===> Epoch[1](300/25343): Loss: 0.3026
lossMSE: 0.008685322478413582 lossSSIM: 0.3064940571784973
===> Epoch[1](350/25343): Loss: 0.3065
lossMSE: 0.005391164682805538 lossSSIM: 0.30530381202697754
===> Epoch[1](400/25343): Loss: 0.3053
lossMSE: 0.006050083786249161 lossSSIM: 0.3122648000717163
===> Epoch[1](450/25343): Loss: 0.3123
lossMSE: 0.00685601308

lossMSE: 0.0013323359889909625 lossSSIM: 0.174993097782135
===> Epoch[1](4150/25343): Loss: 0.1750
lossMSE: 0.005398801527917385 lossSSIM: 0.2711421847343445
===> Epoch[1](4200/25343): Loss: 0.2711
lossMSE: 0.007371341343969107 lossSSIM: 0.32641690969467163
===> Epoch[1](4250/25343): Loss: 0.3264
lossMSE: 0.010562816634774208 lossSSIM: 0.3207828402519226
===> Epoch[1](4300/25343): Loss: 0.3208
lossMSE: 0.0038017970509827137 lossSSIM: 0.2654470205307007
===> Epoch[1](4350/25343): Loss: 0.2654
lossMSE: 0.003838097909465432 lossSSIM: 0.289883553981781
===> Epoch[1](4400/25343): Loss: 0.2899
lossMSE: 0.004177100490778685 lossSSIM: 0.28378826379776
===> Epoch[1](4450/25343): Loss: 0.2838
lossMSE: 0.004667340777814388 lossSSIM: 0.26966673135757446
===> Epoch[1](4500/25343): Loss: 0.2697
lossMSE: 0.005759344436228275 lossSSIM: 0.2981148958206177
===> Epoch[1](4550/25343): Loss: 0.2981
lossMSE: 0.007508253213018179 lossSSIM: 0.3106200098991394
===> Epoch[1](4600/25343): Loss: 0.3106
lossMSE: 0

lossMSE: 0.011174162849783897 lossSSIM: 0.3023800849914551
===> Epoch[1](8300/25343): Loss: 0.3024
lossMSE: 0.004782581236213446 lossSSIM: 0.27064239978790283
===> Epoch[1](8350/25343): Loss: 0.2706
lossMSE: 0.0035644115414470434 lossSSIM: 0.29262685775756836
===> Epoch[1](8400/25343): Loss: 0.2926
lossMSE: 0.0027268542908132076 lossSSIM: 0.2136244773864746
===> Epoch[1](8450/25343): Loss: 0.2136
lossMSE: 0.00994249526411295 lossSSIM: 0.26610320806503296
===> Epoch[1](8500/25343): Loss: 0.2661
lossMSE: 0.010497679002583027 lossSSIM: 0.27487850189208984
===> Epoch[1](8550/25343): Loss: 0.2749
lossMSE: 0.009875316172838211 lossSSIM: 0.2745007276535034
===> Epoch[1](8600/25343): Loss: 0.2745
lossMSE: 0.0031674699857831 lossSSIM: 0.2558705806732178
===> Epoch[1](8650/25343): Loss: 0.2559
lossMSE: 0.008763275109231472 lossSSIM: 0.2662305235862732
===> Epoch[1](8700/25343): Loss: 0.2662
lossMSE: 0.010085110552608967 lossSSIM: 0.2849370837211609
===> Epoch[1](8750/25343): Loss: 0.2849
lossMSE

lossMSE: 0.006917132064700127 lossSSIM: 0.28665560483932495
===> Epoch[1](12400/25343): Loss: 0.2867
lossMSE: 0.003696391126140952 lossSSIM: 0.2981802225112915
===> Epoch[1](12450/25343): Loss: 0.2982
lossMSE: 0.006534097716212273 lossSSIM: 0.2981315851211548
===> Epoch[1](12500/25343): Loss: 0.2981
lossMSE: 0.007227682508528233 lossSSIM: 0.24306046962738037
===> Epoch[1](12550/25343): Loss: 0.2431
lossMSE: 0.010244706645607948 lossSSIM: 0.3180691599845886
===> Epoch[1](12600/25343): Loss: 0.3181
lossMSE: 0.007208121940493584 lossSSIM: 0.250571608543396
===> Epoch[1](12650/25343): Loss: 0.2506
lossMSE: 0.010283863171935081 lossSSIM: 0.33724749088287354
===> Epoch[1](12700/25343): Loss: 0.3372
lossMSE: 0.0034283865243196487 lossSSIM: 0.22425252199172974
===> Epoch[1](12750/25343): Loss: 0.2243
lossMSE: 0.00820926483720541 lossSSIM: 0.19872039556503296
===> Epoch[1](12800/25343): Loss: 0.1987
lossMSE: 0.00685702171176672 lossSSIM: 0.27114230394363403
===> Epoch[1](12850/25343): Loss: 0.2

lossMSE: 0.012458649463951588 lossSSIM: 0.39292508363723755
===> Epoch[1](16500/25343): Loss: 0.3929
lossMSE: 0.003017514944076538 lossSSIM: 0.224432110786438
===> Epoch[1](16550/25343): Loss: 0.2244
lossMSE: 0.0021067336201667786 lossSSIM: 0.22217869758605957
===> Epoch[1](16600/25343): Loss: 0.2222
lossMSE: 0.006263796705752611 lossSSIM: 0.2905658483505249
===> Epoch[1](16650/25343): Loss: 0.2906
lossMSE: 0.012262308970093727 lossSSIM: 0.3049774765968323
===> Epoch[1](16700/25343): Loss: 0.3050
lossMSE: 0.003790598828345537 lossSSIM: 0.21694010496139526
===> Epoch[1](16750/25343): Loss: 0.2169
lossMSE: 0.008462782949209213 lossSSIM: 0.3320164084434509
===> Epoch[1](16800/25343): Loss: 0.3320
lossMSE: 0.010600373148918152 lossSSIM: 0.2958414554595947
===> Epoch[1](16850/25343): Loss: 0.2958
lossMSE: 0.008061017841100693 lossSSIM: 0.25285184383392334
===> Epoch[1](16900/25343): Loss: 0.2529
lossMSE: 0.004542926326394081 lossSSIM: 0.26413583755493164
===> Epoch[1](16950/25343): Loss: 0.

...

lossMSE: 0.00970897451043129 lossSSIM: 0.2735045552253723
===> Epoch[16](25200/25343): Loss: 0.2735
lossMSE: 0.004950063768774271 lossSSIM: 0.20073992013931274
===> Epoch[16](25250/25343): Loss: 0.2007
lossMSE: 0.0030623215716332197 lossSSIM: 0.22867387533187866
===> Epoch[16](25300/25343): Loss: 0.2287
===> Epoch 16 Complete: Avg. Loss: 0.2662
lossMSE: 0.009691321291029453 lossSSIM: 0.2848455309867859
===> Epoch[17](0/25343): Loss: 0.2848
lossMSE: 0.005200336221605539 lossSSIM: 0.26647329330444336
===> Epoch[17](50/25343): Loss: 0.2665
lossMSE: 0.0035946527495980263 lossSSIM: 0.2697519063949585
===> Epoch[17](100/25343): Loss: 0.2698
lossMSE: 0.008671457879245281 lossSSIM: 0.30103373527526855
===> Epoch[17](150/25343): Loss: 0.3010
lossMSE: 0.002991863526403904 lossSSIM: 0.21302998065948486
===> Epoch[17](200/25343): Loss: 0.2130
lossMSE: 0.009162260219454765 lossSSIM: 0.2895205020904541
===> Epoch[17](250/25343): Loss: 0.2895
lossMSE: 0.0037930479738861322 lossSSIM: 0.258731424808502

lossMSE: 0.0012996885925531387 lossSSIM: 0.17107617855072021
===> Epoch[17](3950/25343): Loss: 0.1711
lossMSE: 0.007367430254817009 lossSSIM: 0.28696370124816895
===> Epoch[17](4000/25343): Loss: 0.2870
lossMSE: 0.004006243776530027 lossSSIM: 0.2708933353424072
===> Epoch[17](4050/25343): Loss: 0.2709
lossMSE: 0.006162573117762804 lossSSIM: 0.30762743949890137
===> Epoch[17](4100/25343): Loss: 0.3076
lossMSE: 0.0042126900516450405 lossSSIM: 0.2453862428665161
===> Epoch[17](4150/25343): Loss: 0.2454
lossMSE: 0.010540394112467766 lossSSIM: 0.28735077381134033
===> Epoch[17](4200/25343): Loss: 0.2874
lossMSE: 0.006365621462464333 lossSSIM: 0.27790480852127075
===> Epoch[17](4250/25343): Loss: 0.2779
lossMSE: 0.016181930899620056 lossSSIM: 0.2696552276611328
===> Epoch[17](4300/25343): Loss: 0.2697
lossMSE: 0.0074737947434186935 lossSSIM: 0.24389886856079102
===> Epoch[17](4350/25343): Loss: 0.2439
lossMSE: 0.00542395981028676 lossSSIM: 0.28720349073410034
===> Epoch[17](4400/25343): Loss

lossMSE: 0.005147948861122131 lossSSIM: 0.2168947458267212
===> Epoch[17](8050/25343): Loss: 0.2169
lossMSE: 0.005593175068497658 lossSSIM: 0.2612495422363281
===> Epoch[17](8100/25343): Loss: 0.2612
lossMSE: 0.005069044418632984 lossSSIM: 0.25190669298171997
===> Epoch[17](8150/25343): Loss: 0.2519
lossMSE: 0.005157070234417915 lossSSIM: 0.2657598853111267
===> Epoch[17](8200/25343): Loss: 0.2658
lossMSE: 0.009449172765016556 lossSSIM: 0.2742234468460083
===> Epoch[17](8250/25343): Loss: 0.2742
lossMSE: 0.010254126973450184 lossSSIM: 0.3193349242210388
===> Epoch[17](8300/25343): Loss: 0.3193
lossMSE: 0.0062986877746880054 lossSSIM: 0.22847354412078857
===> Epoch[17](8350/25343): Loss: 0.2285
lossMSE: 0.0033910763449966908 lossSSIM: 0.28629153966903687
===> Epoch[17](8400/25343): Loss: 0.2863
lossMSE: 0.004616491962224245 lossSSIM: 0.26714015007019043
===> Epoch[17](8450/25343): Loss: 0.2671
lossMSE: 0.010939892381429672 lossSSIM: 0.30329614877700806
===> Epoch[17](8500/25343): Loss: 

lossMSE: 0.016023460775613785 lossSSIM: 0.3238595128059387
===> Epoch[17](12150/25343): Loss: 0.3239
lossMSE: 0.010487104766070843 lossSSIM: 0.2985333800315857
===> Epoch[17](12200/25343): Loss: 0.2985
lossMSE: 0.005704643204808235 lossSSIM: 0.2871541380882263
===> Epoch[17](12250/25343): Loss: 0.2872
lossMSE: 0.007955876179039478 lossSSIM: 0.34861284494400024
===> Epoch[17](12300/25343): Loss: 0.3486
lossMSE: 0.005144573748111725 lossSSIM: 0.27986299991607666
===> Epoch[17](12350/25343): Loss: 0.2799
lossMSE: 0.0036009831819683313 lossSSIM: 0.22581690549850464
===> Epoch[17](12400/25343): Loss: 0.2258
lossMSE: 0.0032306313514709473 lossSSIM: 0.22073781490325928
===> Epoch[17](12450/25343): Loss: 0.2207
lossMSE: 0.0033794527407735586 lossSSIM: 0.24694526195526123
===> Epoch[17](12500/25343): Loss: 0.2469
lossMSE: 0.0021556513383984566 lossSSIM: 0.2268708348274231
===> Epoch[17](12550/25343): Loss: 0.2269
lossMSE: 0.007018559612333775 lossSSIM: 0.29726338386535645
===> Epoch[17](12600/2

lossMSE: 0.003238911274820566 lossSSIM: 0.234869122505188
===> Epoch[17](16200/25343): Loss: 0.2349
lossMSE: 0.01201457716524601 lossSSIM: 0.29605257511138916
===> Epoch[17](16250/25343): Loss: 0.2961
lossMSE: 0.004363055340945721 lossSSIM: 0.273720920085907
===> Epoch[17](16300/25343): Loss: 0.2737
lossMSE: 0.002604052424430847 lossSSIM: 0.22795695066452026
===> Epoch[17](16350/25343): Loss: 0.2280
lossMSE: 0.007587186060845852 lossSSIM: 0.22309744358062744
===> Epoch[17](16400/25343): Loss: 0.2231
lossMSE: 0.006681046448647976 lossSSIM: 0.3044918179512024
===> Epoch[17](16450/25343): Loss: 0.3045
lossMSE: 0.014047814533114433 lossSSIM: 0.3389425277709961
===> Epoch[17](16500/25343): Loss: 0.3389
lossMSE: 0.0034712962806224823 lossSSIM: 0.22718751430511475
===> Epoch[17](16550/25343): Loss: 0.2272
lossMSE: 0.0035697147250175476 lossSSIM: 0.22609949111938477
===> Epoch[17](16600/25343): Loss: 0.2261
lossMSE: 0.004252180457115173 lossSSIM: 0.2529698610305786
===> Epoch[17](16650/25343):